### Predict future loan demand or interest rates using time series analysis.

In [5]:
# Import necessary libraries
from fredapi import Fred
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import warnings

# Disable warnings
warnings.filterwarnings('ignore')

# Initialize Fred API with your API key
fred = Fred(api_key='dc7c3ec781759add86587b96ae2c8b0a')

# Fetch U.S. 10-Year Treasury Constant Maturity Rate (DGS10)
data = fred.get_series('DGS10')

# Convert to a pandas DataFrame
data = pd.DataFrame(data, columns=['Interest Rate'])
data.index.name = 'Date'
data.reset_index(inplace=True)

# Display first few rows of the data
print(data.head())

# Plot the interest rates over time
plt.figure(figsize=(10, 6))
plt.plot(data['Date'], data['Interest Rate'], label='10-Year Treasury Rate')
plt.title('10-Year Treasury Constant Maturity Rate Over Time')
plt.xlabel('Date')
plt.ylabel('Interest Rate (%)')
plt.legend()
plt.show()

# Step 1: Check for stationarity with ADF Test
adf_result = adfuller(data['Interest Rate'].dropna())
print(f'ADF Statistic: {adf_result[0]}')
print(f'p-value: {adf_result[1]}')

# Step 2: If p-value > 0.05, the series is non-stationary; differencing the series
if adf_result[1] > 0.05:
    print("The series is non-stationary. Differencing is required.")
    data['Differenced'] = data['Interest Rate'].diff().dropna()
    plt.figure(figsize=(10, 6))
    plt.plot(data['Date'][1:], data['Differenced'], label='Differenced Interest Rate')
    plt.title('Differenced 10-Year Treasury Constant Maturity Rate Over Time')
    plt.xlabel('Date')
    plt.ylabel('Differenced Interest Rate')
    plt.legend()
    plt.show()
else:
    print("The series is stationary. Proceeding with ARIMA model.")

# Step 3: Fit ARIMA model (Assuming p, d, q = 1, 1, 1 for simplicity)
model = ARIMA(data['Interest Rate'], order=(1, 1, 1))
arima_result = model.fit()

# Display ARIMA model summary
print(arima_result.summary())

# Step 4: Plot the fitted values vs actual interest rate data
plt.figure(figsize=(10, 6))
plt.plot(data['Date'], data['Interest Rate'], label='Actual Data')
plt.plot(data['Date'][1:], arima_result.fittedvalues, color='red', label='Fitted ARIMA Model')
plt.title('ARIMA Model Fitted vs Actual Data')
plt.xlabel('Date')
plt.ylabel('Interest Rate (%)')
plt.legend()
plt.show()

# Step 5: Forecast next 12 months using the ARIMA model
forecast = arima_result.forecast(steps=12)

# Create a new date index for the forecasted values
forecast_dates = pd.date_range(start=data['Date'].iloc[-1], periods=12, freq='M')

# Plot the actual data along with the forecasted values
plt.figure(figsize=(10, 6))
plt.plot(data['Date'], data['Interest Rate'], label='Historical Data')
plt.plot(forecast_dates, forecast, label='Forecasted Data', color='red')
plt.title('Forecast for U.S. 10-Year Treasury Rate')
plt.xlabel('Date')
plt.ylabel('Interest Rate (%)')
plt.legend()
plt.show()

# Step 6: Calculate evaluation metrics (MAE, RMSE)
y_true = data['Interest Rate'].dropna()
y_pred = arima_result.fittedvalues

mae = mean_absolute_error(y_true[1:], y_pred)
rmse = np.sqrt(mean_squared_error(y_true[1:], y_pred))

print(f'MAE: {mae:.4f}')
print(f'RMSE: {rmse:.4f}')


AttributeError: 'xml.etree.ElementTree.Element' object has no attribute 'getchildren'